In [ ]:
# This notebook runs from the projects root directory.
import os
PROJECT_ROOT_DIRECTORY = 'senior_project'
while os.path.basename(os.getcwd()) != PROJECT_ROOT_DIRECTORY:
    os.chdir(os.pardir)
print('Current working directory:', os.getcwd())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
from stable_baselines3 import DQN, A2C
from environments.discrete_env_v2 import PortfolioManagementEnv

In [ ]:
# experiment_id = f'EXPERIMENT_1_{int(time.time())}' # TODO
print(experiment_id)

In [ ]:
models_dir = f'experiments/models/{experiment_id}'
log_dir = f'experiments/logs'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [ ]:
# TODO
train_df = pd.read_csv('data\dow_10_returns_train.csv', index_col=0, parse_dates=True)

TICKERS = ['AXP', 'AAPL', 'BA', 'GS', 'INTC', 'JNJ', 'KO', 'NKE', 'PG', 'DIS']
RETURN_COLS = ['RF_RETURN'] + [f'{ticker}_RETURN' for ticker in TICKERS]
FEATURE_COLS = RETURN_COLS

In [ ]:
# TODO
train_env = PortfolioManagementEnv(
    train_df, 
    RETURN_COLS, 
    FEATURE_COLS, 
    window_size = 20,
    episode_length = 180
)

In [ ]:
# TODO
model = A2C(
    'MlpPolicy',
    train_env,
    verbose = 1,
    tensorboard_log = log_dir,
    seed = 5,
    learning_rate = 0.0005,
    drawdown_penalty_weight = 1,
    allocations_in_obs = True,
)

In [ ]:
TIMESTEPS = 10_000 # number of timesteps between saves
for i in tqdm(range(1, 300)):
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=experiment_id)
    model.save(f'{models_dir}/{TIMESTEPS*i}')